In [1]:
import datetime

import polars as pl

import nwec.utility_reporting.arrearages
import nwec.utils.excel
from nwec.constants import DATA

YEAR = 2024
NUM_MONTHS = 12
COLS_PER_MONTH = 4
spreadsheet = DATA / f"utility_reporting/avista/avista_{YEAR}.xlsx"

# Arrearages

In [2]:
arrearages_index = nwec.utils.excel.get_sheet_index_from_name(spreadsheet, "past due balances")
df = pl.read_excel(spreadsheet, sheet_id=arrearages_index, has_header=False)

In [3]:
_, arrearage_start_index = nwec.utils.excel.find_unpromoted_header(df, "past-due balances by customer class")

# Set up arrearages-specific DF with built-in space for the zip code and customer class columns
arrearages = df.select(df.columns[arrearage_start_index : arrearage_start_index + NUM_MONTHS * COLS_PER_MONTH + 2])
arrearages = nwec.utility_reporting.arrearages.normalize_zip_class_cols(df, arrearages)

In [4]:
zip_index = nwec.utils.excel.find_unpromoted_header(arrearages, "Zip Code")
new_columns = arrearages.select(arrearages.columns[2:][:-2]).slice(zip_index[0] - 1, 1).to_dicts()[0]
vintage_cols = arrearages.select(arrearages.columns[2:][:-2]).slice(zip_index[0], 1).to_dicts()[0]
months = list({k: v for k, v in new_columns.items() if v is not None}.values())

for counter, col in enumerate(vintage_cols):
    current_month = months[counter // COLS_PER_MONTH]
    date = datetime.datetime.strptime(current_month, "%Y-%m-%d %H:%M:%S").astimezone(datetime.UTC)
    new_columns[col] = date.strftime("%B %Y")
    new_columns[col] = new_columns[col] + " " + vintage_cols[col]
new_columns = new_columns | {"Zip Code": "Zip Code", "Customer Class": "Customer Class"}

In [5]:
arrearages = arrearages.rename(new_columns).select(list(new_columns.values()))
arrearages = arrearages.filter(~pl.all_horizontal(pl.all().is_null()))
arrearages = arrearages.filter(pl.col("Customer Class").str.contains(r"(?i)resident")).drop(pl.col("Customer Class"))
arrearages = pl.concat(
    [arrearages.select("Zip Code"), arrearages.drop(pl.selectors.matches(r"(?i)zip|customer class"))], how="horizontal"
)
arrearages = arrearages.with_columns([pl.col(col).cast(pl.Float64, strict=False) for col in arrearages.columns[1:]])

In [6]:
arrearages.sum()

Zip Code,January 2024 30 Days,January 2024 60 Days,January 2024 90 Days +,January 2024 Total Arrearages,February 2024 30 Days,February 2024 60 Days,February 2024 90 Days +,February 2024 Total Arrearages,March 2024 30 Days,March 2024 60 Days,March 2024 90 Days +,March 2024 Total Arrearages,April 2024 30 Days,April 2024 60 Days,April 2024 90 Days +,April 2024 Total Arrearages,May 2024 30 Days,May 2024 60 Days,May 2024 90 Days +,May 2024 Total Arrearages,June 2024 30 Days,June 2024 60 Days,June 2024 90 Days +,June 2024 Total Arrearages,July 2024 30 Days,July 2024 60 Days,July 2024 90 Days +,July 2024 Total Arrearages,August 2024 30 Days,August 2024 60 Days,August 2024 90 Days +,August 2024 Total Arrearages,September 2024 30 Days,September 2024 60 Days,September 2024 90 Days +,September 2024 Total Arrearages,October 2024 30 Days,October 2024 60 Days,October 2024 90 Days +,October 2024 Total Arrearages,November 2024 30 Days,November 2024 60 Days,November 2024 90 Days +,November 2024 Total Arrearages,December 2024 30 Days,December 2024 60 Days,December 2024 90 Days +,December 2024 Total Arrearages
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
null,1.9802e6,1.2420e6,2.8916e6,6.1138e6,2.5340e6,1.6214e6,2.9906e6,7.1460e6,3.3460e6,2.3195e6,3.3744e6,9.0400e6,2.4698e6,2046855.3,3.0270e6,7.5437e6,2.2034e6,1.5750e6,2.7109e6,6.4892e6,1.8379e6,1.6149e6,2.5123e6,5.9651e6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# KLI Arrearages
Same code as arrearages, just with a different search string

In [7]:
kli_arrearages_index = nwec.utils.excel.get_sheet_index_from_name(spreadsheet, "past due balances")
df = pl.read_excel(spreadsheet, sheet_id=kli_arrearages_index, has_header=False)

In [8]:
_, kli_arrearage_start_index = nwec.utils.excel.find_unpromoted_header(
    df, "past-due balances for known low-income household"
)

# Set up arrearages-specific DF with built-in space for the zip code and customer class columns
kli_arrearages = df.select(
    df.columns[kli_arrearage_start_index : kli_arrearage_start_index + NUM_MONTHS * COLS_PER_MONTH + 2]
)
kli_arrearages = nwec.utility_reporting.arrearages.normalize_zip_class_cols(df, kli_arrearages)

In [9]:
zip_index = nwec.utils.excel.find_unpromoted_header(kli_arrearages, "Zip Code")
new_columns = kli_arrearages.select(kli_arrearages.columns[2:][:-2]).slice(zip_index[0] - 1, 1).to_dicts()[0]
vintage_cols = kli_arrearages.select(kli_arrearages.columns[2:][:-2]).slice(zip_index[0], 1).to_dicts()[0]
months = list({k: v for k, v in new_columns.items() if v is not None}.values())

for counter, col in enumerate(vintage_cols):
    current_month = months[counter // COLS_PER_MONTH]
    date = datetime.datetime.strptime(current_month, "%Y-%m-%d %H:%M:%S").astimezone(datetime.UTC)
    new_columns[col] = date.strftime("%B %Y")
    new_columns[col] = new_columns[col] + " " + vintage_cols[col]
new_columns = new_columns | {"Zip Code": "Zip Code", "Customer Class": "Customer Class"}

In [10]:
kli_arrearages = kli_arrearages.rename(new_columns).select(list(new_columns.values()))
kli_arrearages = kli_arrearages.filter(~pl.all_horizontal(pl.all().is_null()))
kli_arrearages = kli_arrearages.filter(pl.col("Customer Class").str.contains(r"(?i)resident")).drop(
    pl.col("Customer Class")
)
kli_arrearages = pl.concat(
    [kli_arrearages.select("Zip Code"), kli_arrearages.drop(pl.selectors.matches(r"(?i)zip|customer class"))],
    how="horizontal",
)
kli_arrearages = kli_arrearages.with_columns(
    [pl.col(col).cast(pl.Float64, strict=False) for col in kli_arrearages.columns[1:]]
)

In [11]:
kli_arrearages.sum()

Zip Code,January 2024 30 Days,January 2024 60 Days,January 2024 90 Days +,January 2024 Total Arrearages,February 2024 30 Days,February 2024 60 Days,February 2024 90 Days +,February 2024 Total Arrearages,March 2024 30 Days,March 2024 60 Days,March 2024 90 Days +,March 2024 Total Arrearages,April 2024 30 Days,April 2024 60 Days,April 2024 90 Days +,April 2024 Total Arrearages,May 2024 30 Days,May 2024 60 Days,May 2024 90 Days +,May 2024 Total Arrearages,June 2024 30 Days,June 2024 60 Days,June 2024 90 Days +,June 2024 Total Arrearages,July 2024 30 Days,July 2024 60 Days,July 2024 90 Days +,July 2024 Total Arrearages,August 2024 30 Days,August 2024 60 Days,August 2024 90 Days +,August 2024 Total Arrearages,September 2024 30 Days,September 2024 60 Days,September 2024 90 Days +,September 2024 Total Arrearages,October 2024 30 Days,October 2024 60 Days,October 2024 90 Days +,October 2024 Total Arrearages,November 2024 30 Days,November 2024 60 Days,November 2024 90 Days +,November 2024 Total Arrearages,December 2024 30 Days,December 2024 60 Days,December 2024 90 Days +,December 2024 Total Arrearages
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
null,359026.37,385153.04,1.4268e6,2.1710e6,418069.06,500612.85,1.4692e6,2.3878e6,423795.41,480086.37,1.6715e6,2.5754e6,294259.5,428951.27,1.3864e6,2.1096e6,247913.34,269823.11,1.2266e6,1.7443e6,181198.94,252652.83,1.0977e6,1.5316e6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
